In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import math
from sklearn.utils import shuffle
import tensorflow as tf
import os
import matplotlib.image as mpimg
from scipy.misc import imresize

# Load all driving log data (3 images, angle, throttle, break, speed)
driving_data = pd.read_csv('driving_log1.csv', sep=',', header = None)

# Make lists of only the center image names (to pull image data later) and steering angles
center_image_names = []
angles = []

for x in range(1, len(driving_data)):
    center_image = driving_data.get_value(x, 0)
    angle = driving_data.get_value(x, 3)
    center_image_names.append(center_image)
    angles.append(angle)

# Pull in all images
print('Loading images. This might take a bit.')
images = []
for i in center_image_names:
    image = mpimg.imread(i)
    image = imresize(image, (40, 80, 3))
    images.append(image)

# Make into numpy arrays to feed Keras
images = np.array(images)
angles = np.array(angles)

print('All images loaded.')

# Change to typical training naming conventions
X_train, y_train = images, angles

# Shuffling first, then splitting into training and validation sets
X_train, y_train = shuffle(X_train, y_train)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=23)

# Import necessary items from Keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Convolution2D, Flatten, MaxPooling2D
from keras.layers.normalization import BatchNormalization

# Now for the actual neural network
print('Neural network initializing.')

batch_size = 100
epochs = 15
pool_size = (2, 2)
input_shape = X_train.shape[1:]

model = Sequential()
model.add(BatchNormalization(input_shape=input_shape))

# Convolutional Layer 1 and Dropout
model.add(Convolution2D(64, 3, 3, border_mode='valid', subsample=(1,1)))
model.add(Activation('relu'))
model.add(Dropout(0.2))

# Conv Layer 2
model.add(Convolution2D(32, 3, 3, border_mode='valid', subsample=(1,1)))
model.add(Activation('relu'))

# Conv Layer 3
model.add(Convolution2D(16, 3, 3, border_mode='valid', subsample=(1,1)))
model.add(Activation('relu'))

# Conv Layer 4
model.add(Convolution2D(8, 3, 3, border_mode='valid', subsample=(1,1)))
model.add(Activation('relu'))

# Pooling
model.add(MaxPooling2D(pool_size=pool_size))

# Flatten and Dropout
model.add(Flatten())
model.add(Dropout(0.5))

# Fully Connected Layer 1 and Dropout
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# FC Layer 2
model.add(Dense(64))
model.add(Activation('relu'))

# FC Layer 3
model.add(Dense(32))
model.add(Activation('relu'))

# Final FC Layer - just one output - steering angle
model.add(Dense(1))

# Compiling and training the model
model.compile(metrics=['mean_squared_error'], optimizer='Nadam', loss='mean_squared_error')

model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=epochs, verbose=2, validation_data=(X_val, y_val))

# Save model architecture and weights
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')

# Show summary of model
model.summary()

Loading images. This might take a bit.
All images loaded.


Using TensorFlow backend.


Neural network initializing.


c:\users\teja\miniconda3\lib\site-packages\ipykernel_launcher.py:63: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1), padding="valid")`
c:\users\teja\miniconda3\lib\site-packages\ipykernel_launcher.py:68: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), strides=(1, 1), padding="valid")`
c:\users\teja\miniconda3\lib\site-packages\ipykernel_launcher.py:72: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), strides=(1, 1), padding="valid")`
c:\users\teja\miniconda3\lib\site-packages\ipykernel_launcher.py:76: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), strides=(1, 1), padding="valid")`
c:\users\teja\miniconda3\lib\site-packages\keras\models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 9380 samples, validate on 1043 samples
Epoch 1/15
586s - loss: 0.0210 - mean_squared_error: 0.0210 - val_loss: 0.0176 - val_mean_squared_error: 0.0176
Epoch 2/15
731s - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.0119 - val_mean_squared_error: 0.0119
Epoch 3/15
596s - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.0111 - val_mean_squared_error: 0.0111
Epoch 4/15
555s - loss: 0.0132 - mean_squared_error: 0.0132 - val_loss: 0.0104 - val_mean_squared_error: 0.0104
Epoch 5/15
559s - loss: 0.0127 - mean_squared_error: 0.0127 - val_loss: 0.0097 - val_mean_squared_error: 0.0097
Epoch 6/15
557s - loss: 0.0128 - mean_squared_error: 0.0128 - val_loss: 0.0099 - val_mean_squared_error: 0.0099
Epoch 7/15
613s - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.0097 - val_mean_squared_error: 0.0097
Epoch 8/15
675s - loss: 0.0119 - mean_squared_error: 0.0119 - val_loss: 0.0093 - val_mean_squared_error: 0.0093
Epoch 9/15
566s - loss: 0.0117 - mean_squared_error: 0.0